In [191]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import openpyxl
import geopandas as gpd
import math
import itertools

In [3]:
input_path='../../data/input/dhis2/new_system/'
output_path='../../data/output/sprint3_analysis/'
shapes_path='../../data/shapes/district/districts_17_19.shp'
input_path_old='../../data/input/dhis2/old_system/'

In [4]:
df_Jan = pd.read_csv(input_path+'Data_elements_(Cases)_jan.csv')
df_Feb = pd.read_csv(input_path+'Data_elements_(Cases)_feb.csv')
df_Mar = pd.read_csv(input_path+'Data_elements_(Cases)_march.csv')
df_Apr = pd.read_csv(input_path+'Data_elements_(Cases)_april.csv')

# Some functions I'll use

In [5]:
districts = list(pd.read_csv(input_path+'districts.csv')["Districts"])

# To clean the data downloaded by Nancy in a stacked format

def get_clean_pivot_from_stack(indicator):
    df = pd.DataFrame(districts).reset_index(drop=True)

    Jan = df_Jan[df_Jan['Data']==indicator].drop('Data', axis=1)
    Feb = df_Feb[df_Feb['Data']==indicator].drop('Data', axis=1)
    March = df_Mar[df_Mar['Data']==indicator].drop('Data', axis=1)
    April = df_Apr[df_Apr['Data']==indicator].drop('Data', axis=1)
    
    #print(Jan.shape[0]+ Feb.shape[0]+ March.shape[0] + April.shape[0])

    df_final = Jan.merge(Feb,on='Organisation unit', how='outer').merge(March,on='Organisation unit', how='outer').merge(April,on='Organisation unit', how='outer')
    #df_final.columns = ["Districts", "Jan_"+indicator, "Feb_"+indicator, "Mar_"+indicator, "Apr_"+indicator]
    df_final.set_index('Organisation unit',drop=True, inplace=True)
    columns = [('2020',indicator,"Jan"), ('2020',indicator,"Feb"),('2020',indicator,"Mar"),('2020',indicator,"Apr")]
    df_final.columns=pd.MultiIndex.from_tuples(columns)
    return df_final


In [6]:
shapes = gpd.read_file(shapes_path)

# Creating a dict of names to replace dsirtcit names

def get_district_name_dict (df):
    keys = list(set(df.index).difference(set(shapes['name_19'])))
    values = list(set(shapes['name_19']).difference(set(df.index)))
    district_name = dict(zip(keys, values))
    return district_name 


In [7]:
# build a small fucntion to split the string column name of the data download as pivot 

def split(strng, sep, occ):
    strng = strng.split(sep)
    return sep.join(strng[occ[1]:]), sep.join(strng[:occ[0]]), sep.join(strng[occ[0]:occ[1]])[:3]

In [8]:
# To clean the data downloaded in a pivot format

def get_clean_pivot(df):
    df['district']=df['organisationunitname'].apply(lambda x: x[:-9].upper())
    df.set_index('district',drop=True,inplace=True)
    cols = np.arange(0,4)
    df.drop(df.columns[cols],axis=1,inplace=True)
    df.rename(index=get_district_name_dict(df),inplace=True)
    cols=df.columns
    new_cols=[]
    for col in cols:
        new_cols.append(split(col,' ',[-2,-1]))
    df.columns=pd.MultiIndex.from_tuples(new_cols)
    return df

In [9]:
# To get year on year deltas over a list of months

def get_delta(df,st_year,end_year,indic,st_month,end_month):
    delta=((df[(end_year,indic,end_month)]
            -df[(st_year,indic,st_month)])
           /df[(st_year,indic,st_month)])
    return delta

In [10]:
# To get month on month deltas

def get_yoy_delta(df,indic,period):
    tot_18 = tot_19= tot_20 = np.zeros(135)
    for month in period:
        tot_18= tot_18+list(df[('2018',indic,month)])
        tot_19= tot_19+list(df[('2019',indic,month)])
        tot_20= tot_20+list(df[('2020',indic,month)])
    delta =  (tot_20-tot_19)/tot_19
    return tot_18,tot_19,tot_20,delta

# First batch of data downloaded by nancy of the new system

# Data elements

In [11]:
multi_index=pd.MultiIndex.from_tuples([('Test','test','test')])
data_elements=pd.DataFrame(index=districts,columns=multi_index)

In [12]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = get_clean_pivot_from_stack(indicator)
    data_elements = pd.merge(data_elements, df_months, how='left',left_index=True,right_index=True) 

In [13]:
data_elements

Test                                              2020  \
                  test 033B-TB04. New and Relapse TB cases started on TX   
                  test                                               Jan   
Abim District      NaN                                               NaN   
Adjumani District  NaN                                               NaN   
Agago District     NaN                                               NaN   
Alebtong District  NaN                                               NaN   
Amolatar District  NaN                                               NaN   
...                ...                                               ...   
Soroti District    NaN                                               NaN   
Tororo District    NaN                                               1.0   
Wakiso District    NaN                                               NaN   
Yumbe District     NaN                                               NaN   
Zombo District     NaN                                               1.0   

                                                                         \
                                    033B-RV07. RE Blister of 28 tablets   
                    Feb   Mar   Apr                                 Jan   
Abim District       1.0   1.0   1.0                                 NaN   
Adjumani District   9.0  29.0  11.0                               386.0   
Agago District      9.0  27.0  22.0                                48.0   
Alebtong District  18.0   4.0   7.0                                 NaN   
Amolatar District   1.0   2.0   3.0                               280.0   
...                 ...   ...   ...                                 ...   
Soroti District     NaN   NaN   NaN                               144.0   
Tororo District     6.0  10.0  13.0                                62.0   
Wakiso District    86.0  72.0  24.0                              2628.0   
Yumbe District     26.0  22.0  26.0                                 NaN   
Zombo District     17.0  21.0  28.0                             10224.0   

                                                                               \
                                               033B-CD02a. Dysentery  - Cases   
                       Feb       Mar       Apr                            Jan   
Abim District          4.0   72058.0  235228.0                           11.0   
Adjumani District   7062.0   26775.0   31874.0                            1.0   
Agago District     17483.0   43926.0   10560.0                            NaN   
Alebtong District  14752.0   18584.0   17555.0                            NaN   
Amolatar District   3862.0    1296.0    1368.0                            NaN   
...                    ...       ...       ...                            ...   
Soroti District     2481.0     620.0     477.0                            NaN   
Tororo District     3826.0    9419.0   14208.0                            NaN   
Wakiso District    29494.0   26278.0   23892.0                            NaN   
Yumbe District     94642.0  130562.0   50148.0                            NaN   
Zombo District     22343.0   23724.0   27374.0                            1.0   

                   ...                                              \
                   ... 033B-CD14d. Hepatitis B- Cases Positive       
                   ...                                     Mar Apr   
Abim District      ...                                     NaN NaN   
Adjumani District  ...                                     NaN NaN   
Agago District     ...                                     NaN NaN   
Alebtong District  ...                                     NaN NaN   
Amolatar District  ...                                     NaN NaN   
...                ...                                     ...  ..   
Soroti District    ...                                     NaN NaN   
Tororo District    ...                                     NaN NaN 

In [14]:
data_elements.drop('test',axis=1,inplace=True,level=2)

In [15]:
null_sum1= data_elements.count()/135

# Indicators 

In [16]:
df_Jan = pd.read_csv(input_path+'Indicator_January.csv')
df_Feb = pd.read_csv(input_path+'Indicator_Feb.csv')
df_Mar = pd.read_csv(input_path+'Indicator_march.csv')
df_Apr = pd.read_csv(input_path+'Indicator_April.csv')

In [17]:
districts = df_Feb[df_Feb['Data']=='EPI - BCG doses given']['Organisation unit'].unique()
multi_index=pd.MultiIndex.from_tuples([('Test','test','test')])

In [18]:
data_Indicators=pd.DataFrame(index=districts,columns=multi_index)

In [19]:
indicators = df_Jan['Data'].unique()
for indicator in indicators:
    df_months = get_clean_pivot_from_stack(indicator)
    data_Indicators = pd.merge(data_Indicators, df_months, how='left',left_index=True,right_index=True)

In [20]:
data_Indicators.reset_index(inplace=True)
data_Indicators=data_Indicators.groupby('index',axis=0).mean() # This shouldnt be needed, there might be a bug

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [21]:
null_sum2= data_Indicators.count()/135

# Putting the data together and matching district to shapefiles

In [22]:
# Looking at data quality and null data in one place

null_sum=pd.concat([null_sum1,null_sum2])
null_sum_df = pd.DataFrame(null_sum)
null_sum_df.columns = ['% of data available']
null_sum_df.index.rename(['year','indicator','month'],inplace=True)
null_av_df=null_sum_df.groupby('indicator')['% of data available'].mean()


In [23]:
#Putting both our data together
data=pd.merge(data_Indicators,data_elements,how='inner',left_index=True,right_index=True)

In [24]:
# Using the same name format as in our shapefile

data.rename(index=lambda s: s [:-9],inplace=True)
data.rename(index=str.upper,inplace=True)

In [25]:
data.head()

2020                 \
         105- Total Linked to HIV care (1-4 Yrs) - Female                  
                                                      Jan  Feb  Mar  Apr   
index                                                                      
ABIM                                                  NaN  0.0  0.0  2.0   
ADJUMANI                                              1.0  1.0  NaN  0.0   
AGAGO                                                 0.0  0.0  0.0  1.0   
ALEBTONG                                              NaN  NaN  1.0  NaN   
AMOLATAR                                              0.0  NaN  1.0  NaN   

                                                                        \
         105- Total Linked to HIV care (1-4 Yrs) - Male                  
                                                    Jan  Feb  Mar  Apr   
index                                                                    
ABIM                                                NaN  0.0  0.0  NaN   
ADJUMANI                                            1.0  0.0  3.0  1.0   
AGAGO                                               0.0  2.0  1.0  0.0   
ALEBTONG                                            1.0  NaN  NaN  NaN   
AMOLATAR                                            2.0  NaN  NaN  1.0   

                                                                 ...  \
         105- Total Linked to HIV care (10-14Yrs) - Female       ...   
                                                       Jan  Feb  ...   
index                                                            ...   
ABIM                                                   NaN  NaN  ...   
ADJUMANI                                               0.0  0.0  ...   
AGAGO                                                  0.0  1.0  ...   
ALEBTONG                                               0.0  1.0  ...   
AMOLATAR                                               0.0  NaN  ...   

                                                      \
         033B-CD14d. Hepatitis B- Cases Positive       
                                             Mar Apr   
index                                                  
ABIM                                         NaN NaN   
ADJUMANI                                     NaN NaN   
AGAGO                                        NaN NaN   
ALEBTONG                                     NaN NaN   
AMOLATAR                                     NaN NaN   

                                                                \
         033B-CD13d. Typhoid Fever- Cases Positive               
                                               Jan Feb Mar Apr   
index                                                            
ABIM                                           NaN NaN NaN NaN   
ADJUMANI                                       NaN NaN NaN NaN   
AGAGO                                          NaN NaN NaN NaN   
ALEBTONG                                       NaN NaN NaN NaN   
AMOLATAR                                       NaN NaN NaN NaN   

                                                                        
         033B-HI11b. Adverse Drug Reactions (ADR) - Deaths              
                                                       Jan Feb Mar Apr  
index                                                                   
ABIM                                                   NaN NaN NaN NaN  
ADJUMANI                                               NaN NaN NaN NaN  
AGAGO                                                  NaN NaN NaN NaN  
ALEBTONG                                               NaN NaN NaN NaN  
AMOLATAR                                               NaN NaN NaN NaN  

[5 rows x 1444 columns]

In [26]:
# Modifying the names to extacly fit whats in the shapefile

shapes = gpd.read_file(shapes_path)

In [27]:
data.rename(index=get_district_name_dict(data),inplace=True)

print(list(set(data.index).difference(set(shapes['name_19']))))

[]


# Selecting both relevant and well reported indicators

In [28]:
# Relevant ones, based on my manual selection

target = pd.read_excel(input_path+'Data summary.xlsx',sheet_name='target')

In [29]:
# Well reported ones, based on number of nulls

#null_av_df.to_csv(output_path+'null_values_summary.csv')

with ExcelWriter(input_path+'Data summary.xlsx',mode='a') as writer:
   null_av_df.to_excel(writer,sheet_name='null_values_summary')

In [30]:
#both

target = pd.merge(target,null_av_df,left_on='Var',right_on='indicator',how='left')
target['keep']=(target['% of data available']>0.7)

In [31]:
target_indicators=list(target.loc[target['keep']==1,'Var'])


# Look at EPI data

## Look at the new data

In [130]:
epi_target=['EPI - BCG doses given',
            #'EPI - BCG to MR 1 dropout rate (%)',
            #'EPI - DPT 1 to DPT3 dropout rate (%)',
            #'EPI - HPV1 doses given',
            #'EPI - HPV2 doses given',
            'EPI - PCV 1 doses_Under 1',
            #'EPI - PCV 2 doses_Under 1',
            'EPI - PCV 3 doses_Under 1',
            'EPI - MR 1 doses given',
            #'EPI - MR 2 doses given']
            'EPI - DPT-HepB-HIB 1 doses given',
            'EPI - DPT-HepB-HIB 3 doses given']
            

# HPV1&2 + MR2 excluded due to low or unstable reporting rates
# PCV2 is excluded to include only first and last dose (mirroring DPT HepB)
# Drop out rates excluded because theyinclude consistently negative drop out rates 

In [131]:
new_epi=data.iloc[:, data.columns.get_level_values(1).isin(epi_target)].copy()
#new_epi.reset_index(inplace=True)

In [132]:
# Is the delta over 4 month the best measure?

print(null_sum.loc['2020',epi_target,['Jan','Feb','Mar','Apr']])

# Looks like yes, as the reporting rates seem steady over the period 


year  indicator                         month
2020  EPI - BCG doses given             Jan      1.0
                                        Feb      1.0
                                        Mar      1.0
                                        Apr      1.0
      EPI - PCV 1 doses_Under 1         Jan      1.0
                                        Feb      1.0
                                        Mar      1.0
                                        Apr      1.0
      EPI - PCV 3 doses_Under 1         Jan      1.0
                                        Feb      1.0
                                        Mar      1.0
                                        Apr      1.0
      EPI - DPT-HepB-HIB 1 doses given  Jan      1.0
                                        Feb      1.0
                                        Mar      1.0
                                        Apr      1.0
      EPI - DPT-HepB-HIB 3 doses given  Jan      1.0
                                        Feb      1.0


C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [133]:
for indicator in epi_target:
    new_epi[('2020',indicator,'delta_jan_20_apr_20')]=get_delta(df=new_epi,
                                                        st_year='2020',
                                                        end_year='2020',
                                                        indic=indicator,
                                                        st_month='Jan',
                                                        end_month='Apr')

Weird phenomenon : negative drop out rates 

"Check for inconsistency in related indicators – **An example of such inconsistency is when there is a negative dropout over a full year** between the number of children receiving the first dose of DPT vaccine and the number receiving the third dose of the vaccine.  Another example, as illustrated in Figure 1 above, is inconsistency between the number of children receiving DPT vaccine and the number of children receiving OPV vaccine.  Yet another example is when there is a negative “wastage rate” (discussed below) -- when the number of children reported to have received a vaccine is greater than the number of doses of the vaccine reported to have been consumed."

from this [WHO document](https://www.who.int/healthinfo/FacilityAnalysisGuide_Immunization.pdf?ua=1#page=11)


## Add in the EPI data from the old DHIS2 instance

In [134]:
old_epi=pd.read_csv(input_path_old+'epi_data_2018_19.csv')

In [135]:
# get this to a clean state

old_epi=get_clean_pivot(old_epi)

In [136]:
# drop the period Iam not interested in for now, to be updated if nw data comes in

#dropped_months=['Jun','Jul','Aug','Sep','Oct','Nov','Dec']
#old_epi.drop(dropped_months,axis=1,inplace=True,level=2)

In [137]:
# Use the same naming convention for indicator as the new DHIS2 instance

old_names= list(old_epi.columns.levels[1])

new_names = ['EPI - DPT-HepB-HIB 1 doses given',
            'EPI - DPT-HepB-HIB 2 doses given',
            'EPI - DPT-HepB-HIB 3 doses given',
            'EPI - BCG doses given',
            'EPI - HPV1 doses given',
            'EPI - HPV2 doses given',
            'EPI - HPV3 doses given',
            'EPI - MR 1 doses given',
            'EPI - PCV 1 doses_Under 1',
            'EPI - PCV 2 doses_Under 1',
            'EPI - PCV 3 doses_Under 1']
            
indicator_name = dict(zip(old_names, new_names))
old_epi.columns.set_levels(list(indicator_name.values()),level=1,inplace=True)

indicator_name

{'DPT1 Under1': 'EPI - DPT-HepB-HIB 1 doses given',
 'DPT2 Under1': 'EPI - DPT-HepB-HIB 2 doses given',
 'DPT3 Under1': 'EPI - DPT-HepB-HIB 3 doses given',
 'EPI - BCG doses given': 'EPI - BCG doses given',
 'HPV1': 'EPI - HPV1 doses given',
 'HPV2': 'EPI - HPV2 doses given',
 'HPV3': 'EPI - HPV3 doses given',
 'Measles Under1': 'EPI - MR 1 doses given',
 'PCV 1 Under1': 'EPI - PCV 1 doses_Under 1',
 'PCV 2 Under1': 'EPI - PCV 2 doses_Under 1',
 'PCV 3 Under1': 'EPI - PCV 3 doses_Under 1'}

In [138]:
# get the Jan-Apr deltas for 2019

for indicator in epi_target:
    old_epi[('2019',indicator,'delta_jan_19_apr_19')]=get_delta(df=old_epi,
                                                        st_year='2019',
                                                        end_year='2019',
                                                        indic=indicator,
                                                        st_month='Jan',
                                                        end_month='Apr')

## Pull all in one df

In [209]:
epi_df=pd.merge(new_epi,old_epi,how='inner',left_index=True,right_index=True)
#new_epi.columns=new_epi.columns.map("_".join)

In [210]:
# get the Mar+Apr deltas for 2019 vs 2020
period=['Mar','Apr']

for indicator in epi_target:
    (epi_df[('2018',indicator,'period_total')],
    epi_df[('2019',indicator,'period_total')],
    epi_df[('2020',indicator,'period_total')],
    epi_df[('2020',indicator,'delta_per_19_per_20')]) = get_yoy_delta(epi_df,indicator,period)

In [211]:
# Get an average for all vaccines distributed

epi_total=epi_df.iloc[:, epi_df.columns.get_level_values(2)=='period_total'].groupby(level=0,axis=1).sum().copy()

epi_df[('2018','all vaccines','period_total')]=epi_total['2018']
epi_df[('2019','all vaccines','period_total')]=epi_total['2019']
epi_df[('2020','all vaccines','period_total')]=epi_total['2020']
epi_df[('2020','all vaccines','delta_per_19_per_20')]=(epi_total['2020']-epi_total['2019'])/epi_total['2019']



## Get that to CSV

In [214]:
epi_export_df=epi_df.copy()
epi_export_df.columns=epi_df.columns.map("_".join)
epi_export_df.to_csv(output_path+'epi_map.csv')

In [265]:
indics_to_keep=['EPI - BCG doses given', 'EPI - PCV 1 doses_Under 1',
       'EPI - PCV 3 doses_Under 1', 'EPI - DPT-HepB-HIB 1 doses given',
       'EPI - DPT-HepB-HIB 3 doses given', 'EPI - MR 1 doses given']

all_months=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


epi_stacked_df=pd.melt(epi_df.copy().reset_index(), id_vars='index')

epi_stacked_df.rename({'variable_0':'year',
                       'variable_1':'indicator',
                       'variable_2':'month',
                       'index':'district'},axis=1,inplace=True)


epi_stacked_df= epi_stacked_df[epi_stacked_df.month.isin(all_months)]
epi_stacked_df=epi_stacked_df[epi_stacked_df.indicator.isin(indics_to_keep)]


epi_stacked_df.to_csv(output_path+'epi_map_stacked.csv')

epi_stacked_df.head()

,district,year,indicator,month,value
0,ABIM,2020,EPI - BCG doses given,Jan,354.0
1,ADJUMANI,2020,EPI - BCG doses given,Jan,976.0
2,AGAGO,2020,EPI - BCG doses given,Jan,736.0
3,ALEBTONG,2020,EPI - BCG doses given,Jan,809.0
4,AMOLATAR,2020,EPI - BCG doses given,Jan,440.0


In [266]:
epi_stacked_df.groupby(['year','month']).sum()

value
year month          
2018 Apr    798391.0
     Aug    807887.0
     Dec    702935.0
     Feb    741898.0
     Jan    815293.0
     Jul    789456.0
     Jun    784952.0
     Mar    811024.0
     May    816813.0
     Nov    826085.0
     Oct    896224.0
     Sep    768554.0
2019 Apr    828513.0
     Aug    848445.0
     Dec    713881.0
     Feb    814854.0
     Jan    847759.0
     Jul    853826.0
     Jun    823335.0
     Mar    844550.0
     May    887434.0
     Nov    815457.0
     Oct    813402.0
     Sep    834373.0
2020 Apr    618138.0
     Feb    801543.0
     Jan    848722.0
     Mar    740263.0

# Look at SAM data

## New data

In [267]:
sam_target_a=['NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
              'NUT: Percentage of pregnant women with Hb level <10g/dl or diagnosed with Anaemia at ANC 1st visit',
              'NUT: Percentage of mothers initiating breast feeding within 1st hour after delivery']

sam_target_b=['105-NA02a1. Children that are underweight (W/A ‚â§-2 SD)',
              '105-NA03a1. Identified malnourished clients(<10) this month - MAM using MUAC',
              #'108-NA01a1. Total number of SAM admissions in ITC',
              #'108-NA01b1. Number of new SAM admissions in ITC',
              #'105-NA03d1. Identified malnourished clients(<10) this month - SAM using W/H or L',
              '105-NA03g1. Identified malnourished clients(<10) this month - Severely Malnourished',
              '105-NA04b1. SAM admissions(<10) in OTC this month - New using MUAC',
              #'105-NA04c1. SAM admissions(<10) in OTC this month - New using W/H or L Z-score',
              '105-NA04a1. SAM admissions(<10) in OTC this month - Old']

sam_admission=['105-NA04b1. SAM admissions(<10) in OTC this month - New using MUAC',
              '105-NA04c1. SAM admissions(<10) in OTC this month - New using W/H or L Z-score',
              '105-NA04a1. SAM admissions(<10) in OTC this month - Old']

# MAM stands for moderate acute malnutrition
# The data was download in two batches, one as in tall format with one file for each month, the other as a pivot 

# Many of those indicators not reported in the old dataset, hence dropped looking at old data


In [268]:
sam_target = [*sam_target_b,*sam_target_a]

In [269]:
sam_target

['105-NA02a1. Children that are underweight (W/A ‚â§-2 SD)',
 '105-NA03a1. Identified malnourished clients(<10) this month - MAM using MUAC',
 '105-NA03g1. Identified malnourished clients(<10) this month - Severely Malnourished',
 '105-NA04b1. SAM admissions(<10) in OTC this month - New using MUAC',
 '105-NA04a1. SAM admissions(<10) in OTC this month - Old',
 'NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
 'NUT: Percentage of pregnant women with Hb level <10g/dl or diagnosed with Anaemia at ANC 1st visit',
 'NUT: Percentage of mothers initiating breast feeding within 1st hour after delivery']

### Look at reporting of the new data

In [270]:
# get the data we already formatted earlier

new_sam_a=data.iloc[:, data.columns.get_level_values(1).isin(sam_target_a)].copy()

In [271]:
# get my batch b in  pivot form

new_sam_b=pd.read_csv(input_path+'sam_data_elements_pivot.csv')
new_sam_b=get_clean_pivot(new_sam_b)

In [272]:
dropped_months=['Nov','Dec']
new_sam_b.drop(dropped_months,axis=1,inplace=True,level=2)

### Look at reporting for the new data

In [273]:
null_sum3= new_sam_b.count()/135
null_sum=pd.concat([null_sum,null_sum3])

In [274]:
print(null_sum.loc['2020',sam_target,['Jan','Feb','Mar','Apr']])

year  indicator                                                                            month
2020  NUT: Percentage of children/babies born with low birth weight (<2.5kg)               Jan      0.992593
                                                                                           Feb      1.000000
                                                                                           Mar      1.000000
                                                                                           Apr      0.992593
      NUT: Percentage of mothers initiating breast feeding within 1st hour after delivery  Jan      1.000000
                                                                                                      ...   
      105-NA04b1. SAM admissions(<10) in OTC this month - New using MUAC                   Apr      0.614815
      105-NA04a1. SAM admissions(<10) in OTC this month - Old                              Jan      0.674074
                               

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


### Put together batch a and b

In [275]:
new_sam=pd.merge(new_sam_a,new_sam_b,how='inner',left_index=True, right_index=True)

In [276]:
new_sam

2020  \
         NUT: Percentage of children/babies born with low birth weight (<2.5kg)   
                                                                            Jan   
ABIM                                                   4.60                       
ADJUMANI                                               9.00                       
AGAGO                                                  9.90                       
ALEBTONG                                               7.20                       
AMOLATAR                                               0.72                       
...                                                     ...                       
SOROTI                                                 9.40                       
TORORO                                                 6.10                       
WAKISO                                                 5.40                       
YUMBE                                                  6.80                       
ZOMBO                                                  8.00                       

                           \
                            
          Feb   Mar   Apr   
ABIM      6.4   6.1   6.3   
ADJUMANI  7.0   9.3   9.8   
AGAGO     7.7   9.6   8.0   
ALEBTONG  3.6   7.5   7.7   
AMOLATAR  3.5   4.1   2.7   
...       ...   ...   ...   
SOROTI    6.7   7.9   8.3   
TORORO    5.2   4.6   6.7   
WAKISO    8.0   8.2   6.7   
YUMBE     5.4   7.0   6.0   
ZOMBO     9.1  11.0  27.6   

                                                                                              \
         NUT: Percentage of mothers initiating breast feeding within 1st hour after delivery   
                                                                                         Jan   
ABIM                                                   89.0                                    
ADJUMANI                                              115.0                                    
AGAGO                                                  83.0                                    
ALEBTONG                                               98.9                                    
AMOLATAR                                              105.8                                    
...                                                     ...                                    
SOROTI                                                 88.1                                    
TORORO                                                101.7                                    
WAKISO                                                 86.8                                    
YUMBE                                                  75.9                                    
ZOMBO                                                  99.8                                    

                               \
                                
            Feb    Mar    Apr   
ABIM      100.4   92.1   64.6   
ADJUMANI  107.8   87.1   96.7   
AGAGO      91.7   57.7   92.8   
ALEBTONG   97.8  116.1   98.3   
AMOLATAR   74.2   82.0   88.4   
...         ...    ...    ...   
SOROTI    100.0  101.2  100.6   
TORORO    105.9   67.1   99.7   
WAKISO     89.2  100.5   95.3   
YUMBE      94.0  102.7   94.3   
ZOMBO     103.2   86.8  107.2   

                                                                                                             \
         NUT: Percentage of pregnant women with Hb level <10g/dl or diagnosed with Anaemia at ANC 1st visit   
                                                                                                        Jan   
ABIM                                                   20.6                                                   
ADJUMANI                                               23.9                                                   
AGAGO                                                   9.0                                                   
ALEBTONG                       

In [277]:
# get the Jan-Apr deltas for 2019

for indicator in sam_target:
    new_sam[('2020',indicator,'delta_jan_20_apr_20')]=get_delta(df=new_sam,
                                                        st_year='2020',
                                                        end_year='2020',
                                                        indic=indicator,
                                                        st_month='Jan',
                                                        end_month='Apr')


In [278]:
new_sam.replace(np.inf, np.nan,inplace=True)

In [279]:
# manually excluding 2 outliers 

new_sam.loc['KABALE',('2020','NUT: Percentage of mothers initiating breast feeding within 1st hour after delivery','delta_jan_20_apr_20')]=np.nan
new_sam.loc['BUTAMBALA',('2020','NUT: Percentage of children/babies born with low birth weight (<2.5kg)','delta_jan_20_apr_20')]=np.nan


## Look at old data for LBW

In [280]:
old_sam=pd.read_csv(input_path_old+'sam_partial_data_2018_19.csv')

In [281]:
# get this to a clean state

old_sam=get_clean_pivot(old_sam)

In [282]:
# drop the period Iam not interested in for now, to be updated if nw data comes in

dropped_months=['Jun','Jul','Aug','Sep','Oct','Nov','Dec']
old_sam.drop(dropped_months,axis=1,inplace=True,level=2)

In [283]:
# Use the same naming convention for indicator as the new DHIS2 instance

old_names= list(old_sam.columns.levels[1])

new_names=['NUT: Percentage of mothers initiating breast feeding within 1st hour after delivery',
              'NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
              '105-NA02a1. Children that are underweight (W/A ‚â§-2 SD)']

indicator_name = dict(zip(old_names, new_names))
old_sam.columns.set_levels(list(indicator_name.values()),level=1,inplace=True)

indicator_name

{'Percent of mothers initiating breastfeeding within 1 hour after birth': 'NUT: Percentage of mothers initiating breast feeding within 1st hour after delivery',
 'Percentage of babies born with low birth weight (<2500 g)': 'NUT: Percentage of children/babies born with low birth weight (<2.5kg)',
 'Underweight rate of children under 5': '105-NA02a1. Children that are underweight (W/A ‚â§-2 SD)'}

In [284]:

sam_df=pd.merge(new_sam,old_sam,how='inner',left_index=True, right_index=True)

In [285]:
# get the Mar+Apr deltas for 2019 vs 2020
period=['Mar','Apr']

for indicator in new_names:
    (sam_df[('2018',indicator,'period_total')],
    sam_df[('2019',indicator,'period_total')],
    sam_df[('2020',indicator,'period_total')],
    sam_df[('2020',indicator,'delta_per_19_per_20')]) = get_yoy_delta(sam_df,indicator,period)

period_test=['Jan','Feb']

for indicator in new_names:
    (sam_df[('2018',indicator,'period_total_test')],
    sam_df[('2019',indicator,'period_total_test')],
    sam_df[('2020',indicator,'period_total_test')],
    sam_df[('2020',indicator,'delta_per_test_19_per_20')]) = get_yoy_delta(sam_df,indicator,period_test)

In [286]:
outliers=['KAZO','KIBOGA','KWEEN','BUTAMBALA']

for o in outliers:
    sam_df.loc[o,('2020','NUT: Percentage of children/babies born with low birth weight (<2.5kg)','delta_per_19_per_20')]=np.nan
    sam_df.loc[o,('2019','NUT: Percentage of children/babies born with low birth weight (<2.5kg)','period_total')]=np.nan
    sam_df.loc[o,('2020','NUT: Percentage of children/babies born with low birth weight (<2.5kg)','period_total')]=np.nan
    sam_df.loc[o,('2018','NUT: Percentage of children/babies born with low birth weight (<2.5kg)','period_total')]=np.nan

## Get that to csv

In [287]:
sam_export_df=sam_df.copy()
sam_export_df.columns=sam_df.columns.map("_".join)
sam_export_df.to_csv(output_path+'sam_map.csv')

Note that I am not looking at old data here, because I coul not find teh corresponding indicators execpt for two of them